In [1]:
%load_ext autoreload
%autoreload 2
import imodels
import importlib
import numpy as np
import random
from imblearn.over_sampling import RandomOverSampler

from rulevetting.api.modeling import fit_models

/Volumes/GoogleDrive/My Drive/research/rules/autogluon/core/src/autogluon/core/scheduler/seq_scheduler.py:119: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if searcher is 'auto':
/Volumes/GoogleDrive/My Drive/research/rules/autogluon/core/src/autogluon/core/utils/feature_selection.py:503: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif prune_threshold is 'noise':


# get data

In [5]:
project_id = 'iai_pecarn'
print('getting', project_id)
project_module_name = f'rulevetting.projects.{project_id}.dataset'
module = importlib.import_module(project_module_name)
dset = module.Dataset()
df_train, df_tune, df_test = dset.get_data(load_csvs=True)

getting iai_pecarn


In [6]:
# balance the classes
over_sampler = RandomOverSampler(random_state=42)
df_train, y_train = over_sampler.fit_resample(df_train.drop(columns='outcome'), df_train['outcome'])
df_train['outcome'] = y_train
df_train['outcome'].value_counts()

0    7111
1    7111
Name: outcome, dtype: int64

# fit models

**fit all interpretable models**

In [7]:
np.random.seed(0)
random.seed(0)
predictor = fit_models(df_train, df_tune, interpretable=True)
predictor.leaderboard(df_tune, silent=True, extra_metrics=['accuracy', 'precision', 'recall'])

Presets specified: ['interpretable']
Beginning AutoGluon training ... Time limit = 30s
AutoGluon will save models to "/Volumes/GoogleDrive/My Drive/research/rules/rule-vetting/data/autogluon_cache/"
AutoGluon Version:  0.3.2b20211022
Train Data Rows:    14222
Train Data Columns: 58
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
NumExpr defaulting to 8 threads.
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    7478.85 MB
	Train Data (Original)  Memory Usage: 6.6 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set

Fitting model: OptimalTree ... Training model for up to 14.54s of the 14.54s of remaining time.
	0.9936	 = Validation score   (roc_auc)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: BoostedRules ... Training model for up to 14.44s of the 14.43s of remaining time.
	0.8477	 = Validation score   (roc_auc)
	0.1s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: BoostedRules_2 ... Training model for up to 14.31s of the 14.31s of remaining time.
	0.8816	 = Validation score   (roc_auc)
	0.16s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 29.57s of the 13.97s of remaining time.
	0.9946	 = Validation score   (roc_auc)
	0.76s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 16.99s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/Volumes/GoogleDrive/My Drive/research/rules/rule-vetting/data/autogluon_cache/")


AttributeError: 'GlobalSparseTreeClassifier' object has no attribute 'complexity_'

In [ ]:
predictor.interpretable_models_summary()

In [8]:
predictor.print_interpretable_rules()

Mined rules:
	feature_15 <= 0.5
	feature_11 <= 0.5
	feature_29 <= 10.5
	feature_8 <= 0.5
	feature_8 <= 0.5



**fit uninterpretable models**

In [9]:
np.random.seed(0)
random.seed(0)
predictor = fit_models(df_train, df_tune, interpretable=False)
predictor.leaderboard(df_tune, silent=True, extra_metrics=['roc_auc', 'precision', 'recall'])

Beginning AutoGluon training ... Time limit = 30s
AutoGluon will save models to "/Volumes/GoogleDrive/My Drive/research/rules/rule-vetting/data/autogluon_cache/"
AutoGluon Version:  0.3.2b20211022
Train Data Rows:    14222
Train Data Columns: 58
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    7782.36 MB
	Train Data (Original)  Memory Usage: 6.6 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the feature

,model,score_test,roc_auc,precision,recall,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.867177,0.867177,0.083333,0.119048,0.999630,0.216544,0.117327,17.578399,0.006414,0.001271,1.050449,2,True,9
1,ExtraTreesGini,0.856137,0.856137,0.055556,0.071429,0.999608,0.126682,0.057294,0.873738,0.126682,0.057294,0.873738,1,True,6
2,RandomForestEntr,0.852707,0.852707,0.056604,0.071429,0.999628,0.098611,0.055881,0.792584,0.098611,0.055881,0.792584,1,True,4
3,ExtraTreesEntr,0.845711,0.845711,0.070175,0.095238,0.999628,0.110953,0.061229,0.856951,0.110953,0.061229,0.856951,1,True,7
4,RandomForestGini,0.842618,0.842618,0.039216,0.047619,0.999606,0.103707,0.056066,0.829376,0.103707,0.056066,0.829376,1,True,3
5,CatBoost,0.784905,0.784905,0.147541,0.214286,0.999333,0.007813,0.004109,14.905990,0.007813,0.004109,14.905990,1,True,5
6,NeuralNetFastAI,0.756770,0.756770,0.109589,0.190476,0.998164,0.061565,0.045316,15.718523,0.061565,0.045316,15.718523,1,True,8
7,KNeighborsUnif,0.571308,0.571308,0.028125,0.428571,0.839192,0.033791,0.029693,0.132638,0.033791,0.029693,0.132638,1,True,1
8,KNeighborsDist,0.492893,0.492893,0.025974,0.380952,0.841194,0.039160,0.020763,0.078780,0.039160,0.020763,0.078780,1,True,2


In [10]:
cls = predictor.explain_classification_errors(df_tune)

CorelsClassifier:

   92 / 2409 (positive class / total)
	↓ 



In [12]:
predictor.feature_importance(df_tune)

Computing feature importance via permutation shuffling for 58 features using 1000 rows with 3 shuffle sets...
	41.52s	= Expected runtime (13.84s per shuffle set)
	5.4s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
AbdTenderDegree_None,0.043408,0.002945,0.000765,3,0.060281,0.026535
MOI_Object struck abdomen,0.029197,0.002758,0.001481,3,0.045001,0.013393
Age,0.025516,0.020221,0.080215,3,0.141387,-0.090354
AbdTenderDegree_Mild,0.017431,0.007909,0.031141,3,0.062750,-0.027888
DecrBreathSound_no,0.009694,0.001014,0.001812,3,0.015501,0.003886
AbdDistention_yes,0.009062,0.002909,0.016336,3,0.025731,-0.007606
LtCostalTender,0.007931,0.006648,0.087373,3,0.046024,-0.030161
VomitWretch_no,0.006298,0.007425,0.139766,3,0.048846,-0.036249
DecrBreathSound_yes,0.006272,0.001925,0.014993,3,0.017300,-0.004757
AbdDistention_no,0.006248,0.002627,0.027084,3,0.021298,-0.008802


# examine predictions

In [99]:
# m = sklearn.linear_model.LogisticRegression()
# predictions = m.fit(df_train.drop(columns='outcome'), df_train['outcome']).predict(df_tune.drop(columns='outcome'))
d = df_train
predictions = 0 * d['outcome']
cls, columns = imodels.explain_classification_errors(d.drop(columns='outcome'),
                                                     0 * d['outcome'], d['outcome'],
                                                     classifier=imodels.CorelsRuleListClassifier(c=0.005))

CorelsClassifier:

   7111 / 14222 (positive class / total)
	↓ 
If GCSScore_1                          → 4670 / 11444 (40.8%)
	↓ 
   2441 /  2778	 
	↓ 
If GCSScore_1  & LtCostalTender_0 == 0 →   0 /    0 (0.0%)
	↓ 
   2441 /  2778	 

